<a href="https://colab.research.google.com/github/da-head0/Spark-study/blob/main/Spark_Definitive_Guide_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar -xvf spark-3.2.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

In [2]:
!git clone https://github.com/databricks/Spark-The-Definitive-Guide.git

Cloning into 'Spark-The-Definitive-Guide'...
remote: Enumerating objects: 2035, done.
remote: Total 2035 (delta 0), reused 0 (delta 0), pack-reused 2035
Receiving objects: 100% (2035/2035), 523.88 MiB | 28.72 MiB/s, done.
Resolving deltas: 100% (305/305), done.
Checking out files: 100% (1738/1738), done.


In [3]:
%cd Spark-The-Definitive-Guide

/content/Spark-The-Definitive-Guide


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

## 스파크 애플리케이션의 아키텍처
- 스파크 드라이버 : 스파크 애플리케이션의 운전자 역할. 
  * 애플리케이션의 실행을 제어, 상태 유지
  * 스파크 클러스터(익스큐터의 상태와 태스크)의 상태 정보 유지.
  * 물리적 컴퓨팅 자원 확보
  * 익스큐터 실행을 위해 클러스터 매니저와 통신

- 스파크 익스큐터 : 드라이버가 할당한 태스크를 수행, 상태와 결과를 드라이버에 보고.
- 클러스터 매니저 : 스파크 애플리케이션을 실행할 클러스터 머신 유지. 
  * 물리적 머신에 연결되어 있음. 
  * 애플리케이션 실행 시 클러스터 매니저에게 자원 할당을 요청
  * 애플리케이션이 실행되는 머신 관리
  * 스탠드얼론 클러스터 매니저 / 아파치 메소스 / 하둡 YARN 등

### 실행 모드 
- 클러스터 모드
  * 가장 흔하게 사용되는 방식
  * 클러스터 매니저가 컴파일된 스크립트를 받고 워커 노드에 드라이버와 익스큐터 프로세스를 실행 (스파크 애플리케이션과 관련된 프로세스 유지)
- 클라이언트 모드
  * 스파크 머신 : 스파크 드라이버 프로세스 유지
  * 클러스터 매니저 : 익스큐터 프로세스 유지
  * 스파크 애플리케이션이 클러스터와 무관한 머신에서 동작
- 로컬 모드 
  * 스파크 애플리케이션이 단일 머신에서 실행됨
  * 애플리케이션의 병렬 처리를 위해 단일 머신의 스레드를 활용
  * 스파크를 학습하거나 애플리케이션 테스트, 개발 중인 애플리케이션을 반복적으로 실험하는 용도로 주로 사용됨
  * 운영용 애플리케이션을 실행할 때는 로컬 모드 사용 권장 X

### 스파크 애플리케이션의 생애주기 (스파크 외부)
1. 클라이언트 요청
  * 스파크 애플리케이션 - 컴파일된 JAR나 라이브러리 파일 - 제출
  * 스파크 드라이버 프로세스의 자원을 함께 요청
  * 클러스터 매니저는 노드 중 하나에 드라이버 프로세스를 실행
  * 애플리케이션이 클러스터에서 실행됨
2. 사용자 코드 실행
  * 반드시 스파크 클러스터(드라이버, 익스큐터)를 초기화하는 SparkSession이 포함되어야 함
    * spark-submit을 실행할 때 사용하는 명령행 인수로 익스큐터 수와 설정값 지정
  * 클러스터 매니저가 익스큐터 프로세스 시작, 결과를 응답받아 익스큐터의 위치 + 관련 정보를 드라이버 프로세스로 전송
  * 스파크 클러스터 완성
3. 코드 실행
  * 드라이버와 워커는 코드를 실행하고 데이터를 이동하는 과정에서 서로 통신
  * 드라이버는 각 워커에 태스크 할당
  * 태스크를 할당받은 워커는 태스크의 상태와 성공/실패 여부를 드라이버에 전송
4. 완료
  * 드라이버 프로세스가 성공이나 실패 중의 하나의 상태로 종료
  * 드라이버가 속한 스파크 클러스터의 모든 익스큐터를 종료

### 스파크 애플리케이션의 생애주기 (스파크 내부)
- 스파크 애플리케이션은 하나 이상의 스파크 잡으로 구성됨
- 스레드를 사용해 여러 액션을 병렬로 수행하는 경우가 아니라면 애플리케이션의 스파크 잡은 차례대로 실행됨

1. SparkSession 생성 - 빌더 메서드 사용


In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Word Count")\
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()

In [7]:
#### SparkContext - 스파크 클러스터에 대한 연결
# RDD같은 스파크의 저수준 API를 사용할 수 있음
df1 = spark.range(2, 10000000, 2)
df2 = spark.range(2, 10000000, 4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")

In [8]:
step4.collect() # 개별 스테이지와 태스크로 이루어진 스파크 잡 실행

[Row(sum(id)=2500000000000)]

- 하나의 액션으로 스파크 잡 완료

In [9]:
step4.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(7) HashAggregate(keys=[], functions=[sum(id#8L)])
   +- ShuffleQueryStage 4
      +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#264]
         +- *(6) HashAggregate(keys=[], functions=[partial_sum(id#8L)])
            +- *(6) Project [id#8L]
               +- *(6) SortMergeJoin [id#8L], [id#2L], Inner
                  :- *(4) Sort [id#8L ASC NULLS FIRST], false, 0
                  :  +- AQEShuffleRead coalesced
                  :     +- ShuffleQueryStage 2
                  :        +- Exchange hashpartitioning(id#8L, 200), ENSURE_REQUIREMENTS, [id=#135]
                  :           +- *(3) Project [(id#0L * 5) AS id#8L]
                  :              +- ShuffleQueryStage 0
                  :                 +- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [id=#68]
                  :                    +- *(1) Range (2, 10000000, step=2, splits=2)
                  +- *(5) Sort [id

#### 스파크 잡 - 액션 하나당 스파크 잡 생성, 액션은 항상 결과를 반환
-  스파크 잡은 일련의 스테이지로 나뉨
- 클러스터의 익스큐터 수보다 파티션 크를 더 크게 지정하는 것이 좋음 

#### 파이프라이닝 : 스파크의 주요 최적화 기법
- 노드간의 데이터 이동 없이 각 노드가 데이터를 직접 공급할 수 있는 연산만 모아 태스크의 단일 스테이지로 만들음

#### 셔플 결과 저장 : 스파크가 노드 간 복제를 유발하는 연산을 실행 시 엔진에서 파이프라이닝을 수행하지 못하므로 네트워크 셔플이 발생
- 데이터 전송이 필요한 '소스' 테스크 먼저 수행
- 소스 태스크의 스테이지가 실행되는 동안 *셔플 파일*을 로컬 디스크에 기록
- 이미 셔플된 데이터를 이용해 새로운 잡을 실행하면 '소스'와 관련된 셔플이 다시 실행되지 않음

## 스파크 애플리케이션 개발

- 스파크에는 빌드 개념이 없으며 Pyspark 애플리케이션은 파이썬 스크립트
- 애플리케이션을 실행하려면 클러스터에서 스크립트를 ㅅ힐행하기만 하면 됨

In [10]:
# 스칼라나 자바의 메인 클래스 역할을 하는 파이썬 파일 작성
# SparkSession을 생성하는 실행 가능한 스크립트 파일 만들음
# spark-submit의 main 인수로 지정할 클래스

from __future__ import print_function
if __name__ == '__main__':
    from pyspark.sql import SparkSession
    spark = SparkSession.builder \
        .master("local") \
        .appName("Word Count") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

    print(spark.range(5000).where("id > 500").selectExpr("sum(id)").collect())

[Row(sum(id)=12372250)]


- 실제와 유사한 데이터를 사용해 비즈니스 로직을 꼼꼼하게 테스트해야 함
  - 이런 유형의 테스트에서는 스파크 단위 테스트를 작성하지 않도록 유의
- 별도의 스레드를 사용해 여러 잡을 동시에 실행할 수 있음